### **Causal Attention: hiding future words**

In [8]:
import torch
import torch.nn as nn

In [9]:
words = ['Your', 'journey', 'starts', 'with', 'one', 'step']

inputs = torch.tensor(
  [
   [0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55], # step     (x^6)
  ]
)

d_in = inputs.shape[-1] # embedding dimension
d_out = 2 # selected dimenstion of trainable weights
d_in, d_out

(3, 2)

In [10]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

In [11]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

In [ ]:
quaries = sa_v2.W_query(inputs)
keys = sa_v2.W_query(inputs)
attn_scores = quaries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
attn_weights

tensor([[0.1621, 0.1886, 0.1873, 0.1520, 0.1446, 0.1653],
        [0.1577, 0.1998, 0.1978, 0.1453, 0.1350, 0.1643],
        [0.1578, 0.1994, 0.1974, 0.1456, 0.1354, 0.1644],
        [0.1619, 0.1850, 0.1840, 0.1548, 0.1485, 0.1658],
        [0.1627, 0.1817, 0.1808, 0.1570, 0.1518, 0.1661],
        [0.1604, 0.1907, 0.1893, 0.1511, 0.1432, 0.1653]],
       grad_fn=<SoftmaxBackward0>)

lower and upper triangular matrix. so the `torch.tril` for lower triangular matrix and `torch.triu` for upper triangular matrix
![lower and upper triangular matrix](../images/L16_s3.png)


In [ ]:
# generate a mask for diagonal zeros for causal attention
context_length = attn_scores.shape[0]
# the tril is related to a concept in matrix called upper and lower triangular matrix. tril=lower triangular matrix which mean all values above the diagonal matrix are all zeros
mask_simple = torch.tril(torch.ones(context_length, context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [17]:
masked_simple = attn_weights * mask_simple
masked_simple

tensor([[0.1621, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1577, 0.1998, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1578, 0.1994, 0.1974, 0.0000, 0.0000, 0.0000],
        [0.1619, 0.1850, 0.1840, 0.1548, 0.0000, 0.0000],
        [0.1627, 0.1817, 0.1808, 0.1570, 0.1518, 0.0000],
        [0.1604, 0.1907, 0.1893, 0.1511, 0.1432, 0.1653]],
       grad_fn=<MulBackward0>)

In [25]:
# now the issue is that these weight cannot be our attention weights cause each row has to sum up to 1. so achieve this, we normalise each row
row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
masked_simple_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4411, 0.5589, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2846, 0.3595, 0.3559, 0.0000, 0.0000, 0.0000],
        [0.2361, 0.2699, 0.2683, 0.2257, 0.0000, 0.0000],
        [0.1951, 0.2179, 0.2169, 0.1882, 0.1820, 0.0000],
        [0.1604, 0.1907, 0.1893, 0.1511, 0.1432, 0.1653]],
       grad_fn=<DivBackward0>)

The previous approach result in a data leakage and the reason is explained in the lecture 16 notes

In [26]:
attn_scores

tensor([[0.4775, 0.6915, 0.6818, 0.3861, 0.3155, 0.5048],
        [0.6915, 1.0263, 1.0121, 0.5753, 0.4714, 0.7497],
        [0.6818, 1.0121, 0.9981, 0.5674, 0.4649, 0.7393],
        [0.3861, 0.5753, 0.5674, 0.3227, 0.2645, 0.4203],
        [0.3155, 0.4714, 0.4649, 0.2645, 0.2169, 0.3444],
        [0.5048, 0.7497, 0.7393, 0.4203, 0.3444, 0.5476]],
       grad_fn=<MmBackward0>)

In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1) # start masking one above the main diagonal, so the diagonal itself stays 0.
masked = attn_scores.masked_fill(mask.bool(), -torch.inf) # this line means, where a value is true, replace with -inf(infinity) else replace with actual attention score. we also use infinity because, performing softmax on them, their value becomes 0.000 and also for numerical stability/correctness hence allowing the row value to sum up to 1
masked


tensor([[0.4775,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.6915, 1.0263,   -inf,   -inf,   -inf,   -inf],
        [0.6818, 1.0121, 0.9981,   -inf,   -inf,   -inf],
        [0.3861, 0.5753, 0.5674, 0.3227,   -inf,   -inf],
        [0.3155, 0.4714, 0.4649, 0.2645, 0.2169,   -inf],
        [0.5048, 0.7497, 0.7393, 0.4203, 0.3444, 0.5476]],
       grad_fn=<MaskedFillBackward0>)

now, all we need to do is apply the softmax function to these masked resuls, and we are done.

In [42]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4411, 0.5589, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2846, 0.3595, 0.3559, 0.0000, 0.0000, 0.0000],
        [0.2361, 0.2699, 0.2683, 0.2257, 0.0000, 0.0000],
        [0.1951, 0.2179, 0.2169, 0.1882, 0.1820, 0.0000],
        [0.1604, 0.1907, 0.1893, 0.1511, 0.1432, 0.1653]],
       grad_fn=<SoftmaxBackward0>)

### MASKING ADDITIONAL ATTENTION WEIGHTS WITH DROPOUT

In [45]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(example)
dropout(example)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])


tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])

In [46]:
# applying dropout to attention weights
torch.manual_seed(123)
dropout(attn_weights)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.1178, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.7119, 0.0000, 0.0000, 0.0000],
        [0.4721, 0.5397, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3902, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3814, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)

creating batches


In [47]:
batch = torch.stack((inputs, inputs), dim=0)
batch.shape

torch.Size([2, 6, 3])


#### 🧠 **What `register_buffer` Does**

In PyTorch, `self.register_buffer(name, tensor)` tells the module:

> “Hey, this tensor is part of the **module’s state**, but it is **not a trainable parameter**.”

So the buffer is:

* ✅ **Saved and loaded** when you call `model.state_dict()` or `torch.save(model)`.
* ✅ Moved automatically to GPU/CPU along with the rest of the model (`model.to(device)`).
* ❌ **Not** optimized — i.e., no gradients are computed for it and it's not updated during backpropagation.

---

#### ⚙️ **Why use it for the Mask**

The **causal mask** (the upper-triangular matrix) is **fixed** for a given `context_length`. It doesn’t change during training or inference. So:

* You **don’t want it to be a learnable parameter** like weights.
* But you **do want it to follow the model** when you move it to GPU, save checkpoints, or load the model elsewhere.

Using `register_buffer` is the perfect fit for this situation.

```python
self.register_buffer(
    'mask',
    torch.triu(torch.ones(context_length, context_length), diagonal=1)
)
```

This registers the causal mask as a **non-trainable, persistent part of the model**.

---

#### 📝 Why Not Just Use a Regular Tensor?

If you just did:

```python
self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
```

then:

* It **wouldn’t automatically move to GPU** if you do `model.to('cuda')`.
* It **wouldn’t be included** in `state_dict`, so if you save and load the model later, the mask would be missing.
* You’d have to manually handle device placement and persistence.

By using `register_buffer`, the mask behaves like a **“read-only parameter”** that travels with the model cleanly.

---

#### 🧠 Analogy

Think of:

* `nn.Parameter` → **trainable weight**, changes with backprop.
* `register_buffer` → **fixed tool/resource**, moves with the model but never gets updated.

---

✅ **Summary**:

> `register_buffer` is used here to store the fixed causal mask inside the module so that it moves with the model to GPU, gets saved/loaded with state dict, but **doesn’t get trained**. This is a clean and proper PyTorch pattern for fixed masks or constants.



In [ ]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        # S = QKᵀ/√dₖ 

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose. basically so here we're swaping(transposing) the rows(context length) and the embedding dimension. eg: (2,6,3)->(2,3,6). this is done so that matrix multiplication can happen. the inner dimensions much be the same. eg 2x6x`3` @ 2x`3`x6. that arguements passes to the transpose function are the dimension so which you want to transpose
        
        
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

Absolutely ✅ — this is a **very good place to pause and visualize**, because the
`masked_fill_( self.mask.bool()[:num_tokens, :num_tokens], -torch.inf )`
line is **crucial** in causal attention and often glossed over.

Let’s break it down carefully with a **concrete toy example**.

---

#### 🧠 **Step 1: Imagine the Shapes**

Say:

* `context_length = 6` (maximum supported sequence length during training)
* `num_tokens = 4` (actual number of tokens in this input — maybe it's a shorter sequence)

The registered mask (created in `__init__`) is upper-triangular:

[
\text{mask shape} = (6, 6)
]

```python
mask = torch.triu(torch.ones(6, 6), diagonal=1)
print(mask)
```

This gives:

```
tensor([[0, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0]])
```

* `0` = allowed (past & current tokens)
* `1` = to be masked (future tokens)

---

#### 🧠 **Step 2: Apply `[:num_tokens, :num_tokens]`**

```python
mask[:4, :4]
```

Gives a **top-left submatrix**:

```
tensor([[0, 1, 1, 1],
        [0, 0, 1, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 0]])
```

👉 Why?
Because this particular batch only has **4 tokens**, so we only need to mask within a **4×4 attention score matrix**.
We ignore the extra part (5–6) since it's irrelevant for this forward pass.

---

#### 🧠 **Step 3: Visualize Attn Scores Before Masking**

Suppose `attn_scores` shape = `(batch=1, 4, 4)` (for simplicity):

```
attn_scores =
tensor([[0.5, 0.8, 0.2, 0.1],
        [0.6, 0.4, 0.3, 0.7],
        [0.2, 0.3, 0.9, 0.5],
        [0.1, 0.4, 0.5, 0.8]])
```

---

#### 🧠 **Step 4: Apply `masked_fill_`**

```python
attn_scores.masked_fill_(mask[:4, :4].bool(), -torch.inf)
```

This replaces all positions where the mask has `True` (i.e., `1`) with `-inf`:

```
masked_attn_scores =
tensor([[ 0.5,   -inf,   -inf,   -inf],
        [ 0.6,    0.4,   -inf,   -inf],
        [ 0.2,    0.3,    0.9,   -inf],
        [ 0.1,    0.4,    0.5,    0.8]])
```

✅ Now:

* Row 0 (token 1) → can **only attend to itself** (col 0). Future tokens are masked.
* Row 1 (token 2) → can attend to **tokens 1 & 2**, but not 3 or 4.
* Row 2 (token 3) → can attend to 1, 2, 3 but not 4.
* Row 3 (token 4) → can attend to 1–4 freely (it’s the last one here).

---

#### 🧠 **Step 5: Softmax**

Then applying:

```python
attn_weights = torch.softmax(masked_attn_scores, dim=-1)
```

will produce a **valid probability distribution per token**, ignoring the `-inf` entries (they become zero probability after softmax):

```
attn_weights ≈
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5987, 0.4013, 0.0000, 0.0000],
        [0.2540, 0.3114, 0.4346, 0.0000],
        [0.1753, 0.2662, 0.2904, 0.2681]])
```

---

#### 🔸 **Why `[:num_tokens, :num_tokens]` Matters**

Because sometimes you’ll pad sequences to a **fixed context length** (e.g., 512), but your **actual sequence might be shorter**. If you applied the full 512×512 mask to a 4-token sequence, PyTorch would complain about a **shape mismatch** or unnecessarily mask extra positions.

Using slicing makes the mask **dynamic per batch**, while keeping the **precomputed full-size mask** in the module buffer.

---

#### ✅ Summary

| Component                        | Purpose                                                                       |
| -------------------------------- | ----------------------------------------------------------------------------- |
| `register_buffer('mask', ...)`   | Precompute causal mask once, store it with the model                          |
| `mask[:num_tokens, :num_tokens]` | Slice it to match the actual sequence length this batch                       |
| `masked_fill_`                   | Replace future-token attention scores with `-inf` to zero them out in softmax |
| Effect                           | Enforces causal (autoregressive) behavior cleanly                             |



In [50]:
torch.manual_seed(123)
context_size = batch.shape[1]
ca = CausalAttention(d_in=d_in, d_out=d_out, context_length=context_size, dropout=0.0)
context_vecs = ca(batch)
context_vecs, context_vecs.shape

(tensor([[[-0.4519,  0.2216],
          [-0.5874,  0.0058],
          [-0.6300, -0.0632],
          [-0.5675, -0.0843],
          [-0.5526, -0.0981],
          [-0.5299, -0.1081]],
 
         [[-0.4519,  0.2216],
          [-0.5874,  0.0058],
          [-0.6300, -0.0632],
          [-0.5675, -0.0843],
          [-0.5526, -0.0981],
          [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>),
 torch.Size([2, 6, 2]))